In [1]:
import MySQLdb as mdb
import os
import json
from stop_words import get_stop_words
from nltk import collocations

In [2]:
BASE_DIR = os.path.abspath('..')

with open(os.path.join(BASE_DIR, '.secure.settings.json')) as secret:
    SECRET = json.loads(secret.read())

SECRET_KEY = SECRET["SECRET_KEY"]

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'TEST_CHARSET': 'UTF8',
        'HOST': '',
        'PORT': '3306',
    }
}

DATABASES['default']['NAME'] = SECRET['DEV_DATABASES_NAME']
DATABASES['default']['USER'] = SECRET['DEV_DATABASES_USER']
DATABASES['default']['PASSWORD'] = SECRET['DEV_DATABASES_PASSWORD']

USER = DATABASES['default']['USER']
PASSWORD = DATABASES['default']['PASSWORD']
NAME = DATABASES['default']['NAME']


class Database(object):
    """Класс для общения с базой данных MySQL"""

    def __init__(self):
        """Создать соединение с базой данных.

        В параметрах соединения указывается хост, логин, пароль, название базы данных, кодировка.
        """
        self._connection = mdb.connect('', USER, PASSWORD, NAME, charset='utf8')

    def commit(self):
        self._connection.commit()

    def execute(self, q):
        """Вернуть результат выполнения запроса в виде массива кортежей.

        Каждый кортеж - строка базы данных, сформированная по запросу.
        """
        self.cur = self._connection.cursor()  # mdb.cursors.DictCursor
        self.cur.execute(q)
        res = self.cur.fetchall()
        self.cur.close()
        return res

In [3]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC"'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

In [6]:
req1 = 'SELECT filename FROM `annotator_document` WHERE subcorpus="RULEC"'

db = Database()
file_draft = tuple(i[0] for i in db.execute(req1))

In [18]:
req1 = 'SELECT filename FROM `annotator_document` WHERE subcorpus="RULEC"'

db = Database()
files = tuple(i[0] for i in db.execute(req1))

In [19]:
file_draft = list()
for f in files:
    if 'draft' in f:
        file_draft.append(f)

In [40]:
drafts = list() # то что надо исключить

In [41]:
for n in file_draft:
    db = Database()
    req = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND filename="' + n + '"'
    drafts.append(tuple(i[0] for i in db.execute(req)))

In [25]:
req1 = 'SELECT title FROM `annotator_document` WHERE subcorpus="RULEC"'

db = Database()
titles = tuple(i[0] for i in db.execute(req1))

In [34]:
title_drafts = list()

for t in titles:
    if 'draft' in t:
        if 'draft 1' not in t and '1st draft' not in t and 'first draft' not in t:
            title_drafts.append(t)

In [43]:
for n in title_drafts:
    db = Database()
    req = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND title="' + n + '"'
    drafts.append(tuple(i[0] for i in db.execute(req)))

In [ ]:
req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="FL" AND date1=2012'

db = Database()
rulec = tuple(i[0] for i in db.execute(req1))

fl_end_texts = []
for n in rulec:
    f = open('fl/end/' + str(n) + '.txt', 'w', encoding='utf-8')
    req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
    text = ''
    for i in db.execute(req2):
        text += i[0]
    fl_end_texts.append(text)
    f.write(text)
    f.close()

In [55]:
not_to_use = list()
for dr in drafts:
    for d in dr:
        not_to_use.append(d)

In [108]:
    req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC"'
    db = Database()
    rulec = tuple(i[0] for i in db.execute(req1))

In [66]:
def get_original_texts(bg, year):
    req1 = 'SELECT id FROM `annotator_document` WHERE subcorpus="RULEC" AND language_background="' + bg + '" AND date1={}'.format(year)

    db = Database()
    rulec = tuple(i[0] for i in db.execute(req1))

    texts = []
    for n in rulec:
        if n not in not_to_use:
            f = open(bg.lower() + '/' + str(year) + '/' + str(n) + '.txt', 'w', encoding='utf-8')
            req2 = 'SELECT text FROM `annotator_sentence` WHERE doc_id_id={}'.format(n)
            text = ''
            for i in db.execute(req2):
                text += i[0]
            texts.append(text)
            f.write(text)
            f.close()
    return texts

In [67]:
fl_2009 = get_original_texts('FL', 2009)

In [68]:
fl_2010 = get_original_texts('FL', 2010)

In [69]:
fl_2011 = get_original_texts('FL', 2011)

In [70]:
fl_2012 = get_original_texts('FL', 2012)

In [71]:
hl_2009 = get_original_texts('HL', 2009)

In [72]:
hl_2010 = get_original_texts('HL', 2010)

In [73]:
hl_2011 = get_original_texts('HL', 2011)

In [74]:
hl_2012 = get_original_texts('HL', 2012)

In [77]:
mystem = os.path.join('..', 'mystem')  # путь к mystem

In [84]:
def lemma(bg, year):
    for d, dirs, files in os.walk(bg.lower() + '/' + str(year) + '/'):
        for filename in files:
            os.system(mystem + ' -ndl ' + os.path.join(d, filename) + ' ' + os.path.join(bg.lower(), str(year) + '_lemma', filename))
        print('Done.')

In [82]:
lemma('FL', 2009)

1147.txt
1107.txt
1187.txt
779.txt
2364.txt
632.txt
525.txt
1506.txt
396.txt
1969.txt
3170.txt
2570.txt
1042.txt
1838.txt
434.txt
26.txt
3213.txt
432.txt
110.txt
584.txt
1051.txt
236.txt
1403.txt
312.txt
663.txt
1383.txt
231.txt
873.txt
1530.txt
1511.txt
1546.txt
3030.txt
1759.txt
546.txt
1786.txt
947.txt
1655.txt
3005.txt
2830.txt
1179.txt
2432.txt
2960.txt
1936.txt
963.txt
564.txt
2657.txt
1543.txt
1769.txt
308.txt
758.txt
1699.txt
3128.txt
961.txt
502.txt
2720.txt
727.txt
2329.txt
1814.txt
258.txt
2418.txt
734.txt
201.txt
1348.txt
313.txt
1356.txt
1512.txt
1278.txt
2924.txt
573.txt
253.txt
2068.txt
790.txt
70.txt
3079.txt
1711.txt
877.txt
2995.txt
1184.txt
2955.txt
2554.txt
2567.txt
76.txt
2961.txt
2012.txt
3149.txt
1983.txt
934.txt
2768.txt
3016.txt
939.txt
387.txt
1140.txt
2766.txt
1125.txt
674.txt
1412.txt
114.txt
2926.txt
596.txt
1060.txt
1263.txt
2627.txt
1264.txt
1427.txt
2591.txt
995.txt
2250.txt
1682.txt
2716.txt
944.txt
2741.txt
129.txt
2651.txt
1211.txt
2407.txt
565.txt
14

In [83]:
lemma('FL', 2010)

375.txt
3237.txt
152.txt
2393.txt
243.txt
581.txt
1916.txt
2317.txt
1338.txt
515.txt
289.txt
445.txt
2774.txt
2807.txt
1745.txt
3276.txt
15.txt
1160.txt
1289.txt
697.txt
932.txt
1568.txt
388.txt
2520.txt
2948.txt
1945.txt
2488.txt
906.txt
1532.txt
1091.txt
748.txt
52.txt
1567.txt
1932.txt
2066.txt
360.txt
2937.txt
1376.txt
3188.txt
1743.txt
1513.txt
2130.txt
1395.txt
1025.txt
2885.txt
2917.txt
2268.txt
690.txt
1964.txt
1941.txt
795.txt
1300.txt
2072.txt
1763.txt
322.txt
2735.txt
420.txt
390.txt
504.txt
2612.txt
1710.txt
994.txt
2336.txt
3048.txt
326.txt
2206.txt
1705.txt
3257.txt
1366.txt
2001.txt
840.txt
2084.txt
2928.txt
2571.txt
2622.txt
2356.txt
2665.txt
595.txt
2215.txt
2731.txt
926.txt
540.txt
3049.txt
44.txt
1805.txt
3256.txt
1725.txt
1285.txt
2431.txt
615.txt
1984.txt
2015.txt
628.txt
1367.txt
591.txt
714.txt
2405.txt
2600.txt
1090.txt
2979.txt
678.txt
904.txt
530.txt
2645.txt
429.txt
977.txt
1847.txt
400.txt
3073.txt
487.txt
1958.txt
692.txt
1007.txt
2273.txt
2426.txt
2381.txt

In [85]:
lemma('FL', 2011)

Done.


In [86]:
lemma('FL', 2012)

Done.


In [87]:
lemma('HL', 2009)

Done.


In [88]:
lemma('HL', 2010)

Done.


In [89]:
lemma('HL', 2011)

Done.


In [90]:
lemma('HL', 2012)

Done.


In [94]:
def get_lemma_texts(bg, year):
    lemma_texts = []

    for d, dirs, files in os.walk(bg + '/' + str(year) + '_lemma/'):
        for filename in files:
            lemma_texts.append(open(bg + '/' + str(year) + '_lemma/' + filename, 'r', encoding='utf-8').read())  
    
    return lemma_texts

In [95]:
fl_2009_lemma = get_lemma_texts('fl', 2009)

In [99]:
fl_2010_lemma = get_lemma_texts('fl', 2010)

In [101]:
fl_2011_lemma = get_lemma_texts('fl', 2011)

In [102]:
fl_2012_lemma = get_lemma_texts('fl', 2012)

In [116]:
stop_words = ['а',
 'в',
 'г',
 'е',
 'ж',
 'и',
 'к',
 'м',
 'о',
 'с',
 'т',
 'у',
 'я',
 'бы',
 'во',
 'вы',
 'да',
 'до',
 'ее',
 'ей',
 'ею',
 'её',
 'же',
 'за',
 'из',
 'им',
 'их',
 'ли',
 'мы',
 'на',
 'не',
 'ни',
 'но',
 'ну',
 'нх',
 'об',
 'он',
 'от',
 'по',
 'со',
 'та',
 'те',
 'то',
 'ту',
 'ты',
 'уж',
 'без',
 'вам',
 'вас',
 'ваш',
 'где',
 'два',
 'две',
 'дел',
 'для',
 'его',
 'ему',
 'еще',
 'ещё',
 'или',
 'ими',
 'имя',
 'как',
 'кем',
 'ком',
 'кто',
 'мне',
 'мог',
 'мож',
 'мои',
 'мой',
 'мор',
 'моя',
 'моё',
 'над',
 'нам',
 'нас',
 'наш',
 'нее',
 'ней',
 'нем',
 'нет',
 'нею',
 'неё',
 'них',
 'она',
 'они',
 'оно',
 'под',
 'пор',
 'при',
 'про',
 'раз',
 'сам',
 'сих',
 'так',
 'там',
 'тем',
 'тех',
 'том',
 'тот',
 'тою',
 'три',
 'тут',
 'уже',
 'чем',
 'что',
 'эта',
 'эти',
 'это',
 'эту',
 'алло',
 'бывь',
 'вами',
 'ваша',
 'ваше',
 'ваши',
 'ведь',
 'вниз',
 'всем',
 'всех',
 'всею',
 'года',
 'году',
 'даже',
 'двух',
 'если',
 'есть',
 'зато',
 'кого',
 'кому',
 'куда',
 'лишь',
 'мало',
 'меля',
 'меня',
 'мимо',
 'мной',
 'мною',
 'нами',
 'наша',
 'наше',
 'наши',
 'него',
 'нему',
 'ниже',
 'ними',
 'один',
 'пока',
 'пора',
 'пять',
 'рано',
 'семь',
 'стал',
 'суть',
 'твой',
 'твоя',
 'твоё',
 'тебе',
 'тебя',
 'теми',
 'того',
 'тоже',
 'тому',
 'туда',
 'хоть',
 'хотя',
 'чаще',
 'чего',
 'чему',
 'чтоб',
 'чуть',
 'этим',
 'этих',
 'этой',
 'этом',
 'этот',
 'более',
 'будто',
 'вверх',
 'вдали',
 'вдруг',
 'везде',
 'внизу',
 'всего',
 'всеми',
 'всему',
 'всюду',
 'давно',
 'даром',
 'друго',
 'затем',
 'зачем',
 'здесь',
 'какая',
 'какой',
 'когда',
 'кроме',
 'лучше',
 'между',
 'менее',
 'много',
 'можхо',
 'назад',
 'низко',
 'одной',
 'около',
 'опять',
 'очень',
 'перед',
 'позже',
 'после',
 'потом',
 'почти',
 'пятый',
 'разве',
 'рядом',
 'самим',
 'самих',
 'самой',
 'самом',
 'своей',
 'своих',
 'сеаой',
 'снова',
 'собой',
 'собою',
 'такая',
 'также',
 'такие',
 'такое',
 'такой',
 'тобой',
 'тобою',
 'тогда',
 'тысяч',
 'часто',
 'через',
 'чтобы',
 'шесть',
 'этими',
 'этого',
 'этому',
 'близко',
 'больше',
 'вокруг',
 'восемь',
 'всегда',
 'второй',
 'далеко',
 'дальше',
 'девять',
 'десять',
 'другая',
 'другие',
 'других',
 'другое',
 'другой',
 'именно',
 'иногда',
 'каждая',
 'каждое',
 'каждые',
 'каждый',
 'кругом',
 'меньше',
 'нельзя',
 'нибудь',
 'никуда',
 'ничего',
 'обычно',
 'однако',
 'одного',
 'отсюда',
 'первый',
 'потому',
 'почему',
 'просто',
 'против',
 'раньше',
 'самими',
 'самого',
 'самому',
 'своего',
 'сейчас',
 'совсем',
 'теперь',
 'только',
 'третий',
 'хорошо',
 'четыре',
 'шестой',
 'восьмой',
 'впрочем',
 'времени',
 'девятый',
 'десятый',
 'конечно',
 'которая',
 'которой',
 'которые',
 'который',
 'которых',
 'наверху',
 'наконец',
 'недавно',
 'немного',
 'нередко',
 'никогда',
 'однажды',
 'посреди',
 'седьмой',
 'сколько',
 'слишком',
 'сначала',
 'спасибо',
 'двадцать',
 'довольно',
 'которого',
 'наиболее',
 'недалеко',
 'особенно',
 'отовсюду',
 'двадцатый',
 'миллионов',
 'несколько',
 'процентов',
 'четвертый',
 'двенадцать',
 'непрерывно',
 'пожалуйста',
 'пятнадцать',
 'семнадцать',
 'тринадцать',
 'двенадцатый',
 'одиннадцать',
 'пятнадцатый',
 'семнадцатый',
 'тринадцатый',
 'шестнадцать',
 'восемнадцать',
 'девятнадцать',
 'одиннадцатый',
 'четырнадцать',
 'шестнадцатый',
 'восемнадцатый',
 'девятнадцатый',
 'четырнадцатый']

for word in ['свой', '&lt;', '«', '–', '-', '(', ')', '»', 'lt', '', 
             'из-за', 'gt', 'какой-то', 'д', 'name', 'ого', 'ом']:
    stop_words.append(word)

In [117]:
def freq_stop(fname, meta, texts):
    dic = {}
    number = 0 # сколько всего слов
    
    for text in texts:
        words = text.split()
        for indx, word in enumerate(words):
            number += 1
            word = word.lower()
            words[indx] = word.strip('.,!@?#$%^&*():"№;%=-,')
            if words[indx] not in stop_words:
                if words[indx] not in dic:
                    dic[words[indx]] = 1
                else:
                    dic[words[indx]] += 1
                    
    f = open('stats/' + fname, 'w', encoding='utf-8')
    f.write(meta + '\n')

    i = 0
    for w in sorted(dic, key=dic.get, reverse=True):
        if i < 100:
            f.write(w + ' ' + str(dic[w]) + ' ' + str(round(dic[w] / number * 1000000, 2)) + '\n')
        else:
            break
        i += 1
    print(number)
        
    f.close()

In [118]:
freq_stop('fl_2009_lemma.txt', 'L2 speakers, year 2009, frequent lemmas', fl_2009_lemma)

30837


In [119]:
freq_stop('fl_2010_lemma.txt', 'L2 speakers, year 2010, frequent lemmas', fl_2010_lemma)

56585


In [120]:
freq_stop('fl_2011_lemma.txt', 'L2 speakers, year 2011, frequent lemmas', fl_2011_lemma)

75356


In [121]:
freq_stop('fl_2012_lemma.txt', 'L2 speakers, year 2012, frequent lemmas', fl_2012_lemma)

93849


In [122]:
hl_2009_lemma = get_lemma_texts('hl', 2009)

In [123]:
hl_2010_lemma = get_lemma_texts('hl', 2010)

In [124]:
hl_2011_lemma = get_lemma_texts('hl', 2011)

In [125]:
hl_2012_lemma = get_lemma_texts('hl', 2012)

In [128]:
freq_stop('hl_2009_lemma.txt', 'Heritage speakers, year 2009, frequent lemmas', hl_2009_lemma)

90971


In [129]:
freq_stop('hl_2010_lemma.txt', 'Heritage speakers, year 2010, frequent lemmas', hl_2010_lemma)

49061


In [130]:
freq_stop('hl_2011_lemma.txt', 'Heritage speakers, year 2011, frequent lemmas', hl_2011_lemma)

34531


In [131]:
freq_stop('hl_2012_lemma.txt', 'Heritage speakers, year 2012, frequent lemmas', hl_2012_lemma)

39798


In [145]:
def get_common(bg):
    lem_09 = open('stats/' + bg + '_2009_lemma.txt', 'r', encoding='utf-8').readlines()
    lem_10 = open('stats/' + bg + '_2010_lemma.txt', 'r', encoding='utf-8').readlines()
    lem_11 = open('stats/' + bg + '_2011_lemma.txt', 'r', encoding='utf-8').readlines()
    lem_12 = open('stats/' + bg + '_2012_lemma.txt', 'r', encoding='utf-8').readlines()
    
    del lem_09[0]
    del lem_10[0]
    del lem_11[0]
    del lem_12[0]
    
    dict_09 = dict()
    dict_10 = dict()
    dict_11 = dict()
    dict_12 = dict()
    
    for indx, el in enumerate(lem_09):
        dict_09[el.split(' ')[0]] = el.split(' ')[1:]
    for indx, el in enumerate(lem_10):
        dict_10[el.split(' ')[0]] = el.split(' ')[1:]
    for indx, el in enumerate(lem_11):
        dict_11[el.split(' ')[0]] = el.split(' ')[1:]
    for indx, el in enumerate(lem_12):
        dict_12[el.split(' ')[0]] = el.split(' ')[1:]
        
    common = set(dict_09.keys()) & set(dict_10.keys()) & set(dict_11.keys()) & set(dict_12.keys())
    
    fw = open('stats/' + bg + '_common.txt', 'w', encoding='utf-8')
    for c in common:
        fw.write(c + '\n')
        fw.write(dict_09[c][0] + ' ' + dict_09[c][1])
        fw.write(dict_10[c][0] + ' ' + dict_10[c][1])
        fw.write(dict_11[c][0] + ' ' + dict_11[c][1])
        fw.write(dict_12[c][0] + ' ' + dict_12[c][1])
        fw.write('\n')   
        
    print(len(common))

In [146]:
get_common('fl')

56


In [147]:
get_common('hl')

49


In [148]:
from lexicalrichness import LexicalRichness

In [149]:
def lexical_diversity(texts):

    lemmed = []

    for text in texts:
        words = text.split()
        lemmed += words
    
    lex = LexicalRichness(' '.join(lemmed))
        
    print('TTR: ', str(lex.ttr))
    print('MTLD: ', str(lex.mtld(threshold=0.72)))

In [150]:
lexical_diversity(fl_2009_lemma)

TTR:  0.11511771191387249
MTLD:  89.44873582171937


In [151]:
lexical_diversity(fl_2010_lemma)

TTR:  0.11552531589643898
MTLD:  93.37838463248991


In [152]:
lexical_diversity(fl_2011_lemma)

TTR:  0.09161723529333705
MTLD:  95.28324389647383


In [153]:
lexical_diversity(fl_2012_lemma)

TTR:  0.07826318593500266
MTLD:  96.13224633260991


In [154]:
lexical_diversity(hl_2009_lemma)

TTR:  0.07951809877656008
MTLD:  116.74225502323321


In [155]:
lexical_diversity(hl_2010_lemma)

TTR:  0.12280129629252186
MTLD:  111.71422848629533


In [156]:
lexical_diversity(hl_2011_lemma)

TTR:  0.1386829607320746
MTLD:  106.6758381833841


In [157]:
lexical_diversity(hl_2012_lemma)

TTR:  0.12337303382079501
MTLD:  107.32745243953417
